# Applied Data Science Capstone Week 3 - Assignment:Segmenting and Clustering Neighborhoods in Toronto

In [165]:
# First, we import the libraries:
import requests
import lxml.html as lh
import pandas as pd

In [184]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

# To manage de contents of the webpage, we will create page:
page = requests.get(url)

# We will save the contents of the website:
doc = lh.fromstring(page.content)

# Now we parse the data:
tr_elements = doc.xpath('//tr')

# We check the length of the first 12 rows:
[len(T) for T in tr_elements[:12]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [212]:
#For each row we will save the header in an empty list:
col=[]
i=0


for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print ('%d:"%s"'%(i,name))
    col.append((name,[]))

1:"Postcode"
2:"Borough"
3:"Neighbourhood
"


In [213]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 3, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [214]:
# We now check the length of each column, the should all be equal:
[len(C) for (title,C) in col]

[287, 287, 287]

In [215]:
# We create the dataframe:

Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

# Access the top 5 rows of the data frame 
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n


In [216]:

# We delete the '\n':
df = df.replace('\n',' ', regex=True)
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [220]:
# We delete all the 'Not assigned' boroughs:

df.drop(df.index[df['Borough'] == 'Not assigned'], inplace = True)

# Reset the index and dropping the previous index
df = df.reset_index(drop=True)

df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


In [221]:
# We order the dataframe:
df.columns = ['Postcode','Borough', 'Neighbourhood']

cols = df.columns.tolist()
cols

cols = cols[-1:] + cols[:-1]

df = df[cols]

In [223]:

# We join similar Postcodes and boroughs:

df = df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(','.join).reset_index()
df.columns = ['Postcode','Borough','Neighbourhood']
df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge ,Malvern"
1,M1C,Scarborough,"Highland Creek ,Rouge Hill ,Port Union"
2,M1E,Scarborough,"Guildwood ,Morningside ,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [224]:
# We trim spaces at the start of the strings:
df['Neighbourhood'] = df['Neighbourhood'].str.strip()

#If Neighbourhood is 'Not assigned' we will assign 'Borough':
df.loc[df['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = df['Borough']

In [225]:
# Check the shape of the data frame
df.shape

(103, 3)

# Latitude and Longitude of Ontario, Canada

In [226]:
# We read the csv provided:

import pandas as pd
import numpy as np 

link = "http://cocl.us/Geospatial_data"
df1 = pd.read_csv(link)

df1.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [177]:
df1.shape

(103, 3)

In [227]:
# We change the name from 'Postal Code' to 'Postocode' to join the two dataframes:

df1.columns = ['Postcode','Latitude','Longitude']

cols = df1.columns.tolist()
cols

['Postcode', 'Latitude', 'Longitude']

In [228]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge ,Malvern"
1,M1C,Scarborough,"Highland Creek ,Rouge Hill ,Port Union"
2,M1E,Scarborough,"Guildwood ,Morningside ,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [229]:
df_LatLo = pd.merge(df, df1, on='Postcode')
df_new.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge ,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek ,Rouge Hill ,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood ,Morningside ,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
